In [64]:
# Dependencies
import json
import pprint
import requests
import sys
import pandas as pd
import numpy as np
import time

#api keys
from config_api import GEOCODE_API_KEY, YELP_API_KEY

#Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


In [53]:
#references
#from lessons Evr_Tweet_Breaker #01-Lessons/07-Social-Analytics/01-Getting-Twitter-Data/04-Evr_Break_Tweet_JSON/Solved/Evr_Tweet_Breaker.ipynb
#from homework and articles s.o.f.
#https://stackoverflow.com/questions/17518937/saving-a-json-file-to-computer-python
#https://stackoverflow.com/questions/14114729/save-a-large-file-using-the-python-requests-library
#https://github.com/Yelp/yelp-python



In [3]:
#file operations

In [50]:
#functions
def load_json(jsonfile):
    """Load JSON from a file"""
    with open(jsonfile) as file_handle:
        return json.load(file_handle)
    
def milestometers(num_miles):
    #mtm = 1.609 * num_miles
    mtm = 1609.34 * num_miles
    mtm = int(mtm)
    return mtm

def meterstomiles(num_meters):
    mtm = num_meters / 1609.34
    mtm = round(mtm,2)
    return mtm

def getyelpreview(yelpid,API_KEY):
    review_url = f'https://api.yelp.com/v3/businesses/{yelpid}/reviews'
    review_headers = {'Authorization': 'Bearer {}'.format(API_KEY),}

    try:
        review_response = requests.get(review_url, headers=review_headers)
        review_responsetype = review_response.status_code
    
        review_data = review_response.json()
        
        #--testing--------->
        #print(response)
        #print(type(response.text))
        #print(response.text[:1000])
        #print(review_data)
        #<----end testing
        
        return review_data
    
    except:
        #print('EXCEPTION')
        return None

def getyelpbusiness(term,location,lat,lng,SEARCH_LIMIT,SEARCH_RADIUS,API_KEY):
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
        'Authorization': 'Bearer {}'.format(API_KEY),
            }

    url_params = {
                'term': term.replace(' ', '+'),
                #'location': location.replace(' ', '+'),
                'latitude': lat,
                'longitude': lng,
                'radius': SEARCH_RADIUS,
                'limit': SEARCH_LIMIT
            }

    try:
        business_response = requests.get(url, headers=headers, params=url_params)
        business_data = business_response.json()
        
        #--testing--------->
        #print(response)
        #print(type(response.text))
        #print(response.text[:1000])
        #print(review_data)
        #<----end testing
        
        return business_data
    
    except:
        #print('EXCEPTION')
        return None

In [5]:
#logic 


In [7]:
#get coordinates by location name


origin_city = "Irvine,California"

#replacing spaces for HTML Encoding?
origin_city = origin_city.replace(" ","%20")

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
location_name = "University of California Irvine"
input_name = location_name.replace(" ","%20")

input_url = "input=" + input_name
input_fields = "&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry"
key_info = "&key="
target_url = f"{base_url}{input_url}{input_fields}{key_info}{GEOCODE_API_KEY}"


In [8]:
# Run a request to endpoint and convert result to json
uci_start_info = requests.get(target_url).json()

# Print the json
print(uci_start_info)

{'candidates': [{'formatted_address': 'Irvine, CA 92697, USA', 'geometry': {'location': {'lat': 33.6404952, 'lng': -117.8442962}, 'viewport': {'northeast': {'lat': 33.66208720000001, 'lng': -117.81300975}, 'southwest': {'lat': 33.625382, 'lng': -117.86935515}}}, 'name': 'University of California Irvine', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103012318601018077697/photos">Sy Guo</a>'], 'photo_reference': 'CmRaAAAAa9ovuA54OAyYVDpBA-InNyH2Wsk4u1Ep__JYDuxN96LMilzr5caduT5kyLCuM6fgSa4d3YT8o2kDdRCqxukz3I4hi9NQwuf5n4blO7r4HbAWD_qw4f9BlFYQMc7Ig6QAEhDpMtyinwOZ0_ONP8jHZVX9GhQgHnif-1ilfjDOSK7qTe8qoQCAbQ', 'width': 4032}], 'rating': 4.2}], 'debug_log': {'line': []}, 'status': 'OK'}


In [9]:
#save json data
file_path = "data/"
file_name = "00_uci_start_latlon.json"
file_save = file_path + file_name

#from homework and articles s.o.f.
#https://stackoverflow.com/questions/17518937/saving-a-json-file-to-computer-python
#https://stackoverflow.com/questions/14114729/save-a-large-file-using-the-python-requests-library

#saving the file in case i get blocked due to over limit warnings again
with open(file_save, "w") as f:
    json.dump(uci_start_info, f)


In [10]:
#print out json file
print(json.dumps(uci_start_info, indent=4, sort_keys=True))

{
    "candidates": [
        {
            "formatted_address": "Irvine, CA 92697, USA",
            "geometry": {
                "location": {
                    "lat": 33.6404952,
                    "lng": -117.8442962
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.66208720000001,
                        "lng": -117.81300975
                    },
                    "southwest": {
                        "lat": 33.625382,
                        "lng": -117.86935515
                    }
                }
            },
            "name": "University of California Irvine",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103012318601018077697/photos\">Sy Guo</a>"
                    ],
                    "photo_reference": "CmRaAAAAa9ovuA54OAyYVDpBA-InNyH2Wsk4u1Ep__

In [11]:
#get important information
location_info = uci_start_info["candidates"]
location_info

[{'formatted_address': 'Irvine, CA 92697, USA',
  'geometry': {'location': {'lat': 33.6404952, 'lng': -117.8442962},
   'viewport': {'northeast': {'lat': 33.66208720000001, 'lng': -117.81300975},
    'southwest': {'lat': 33.625382, 'lng': -117.86935515}}},
  'name': 'University of California Irvine',
  'photos': [{'height': 3024,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103012318601018077697/photos">Sy Guo</a>'],
    'photo_reference': 'CmRaAAAAa9ovuA54OAyYVDpBA-InNyH2Wsk4u1Ep__JYDuxN96LMilzr5caduT5kyLCuM6fgSa4d3YT8o2kDdRCqxukz3I4hi9NQwuf5n4blO7r4HbAWD_qw4f9BlFYQMc7Ig6QAEhDpMtyinwOZ0_ONP8jHZVX9GhQgHnif-1ilfjDOSK7qTe8qoQCAbQ',
    'width': 4032}],
  'rating': 4.2}]

In [12]:
uci_coords = location_info[0]["geometry"]["location"]
#uci_coords

uci_lat = uci_coords["lat"]
uci_lng = uci_coords["lng"]

print(f"{uci_coords}")

print(f'LAT: {uci_lat} - LNG: {uci_lng}')

{'lat': 33.6404952, 'lng': -117.8442962}
LAT: 33.6404952 - LNG: -117.8442962


In [14]:
#Reverse Geocode for accurate address

#https://maps.googleapis.com/maps/api/geocode/json?latlng=33.6404952,-117.8442962&key=
addr_url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={uci_lat},{uci_lng}&key={GEOCODE_API_KEY}"
#print(addr_url)

In [15]:
#Run a request to endpoint and convert result to json
uci_addr_info = requests.get(addr_url).json()

# Print the json
#print(uci_addr_info)

In [16]:
#print out json file
print(json.dumps(uci_addr_info, indent=4, sort_keys=True))

{
    "plus_code": {
        "compound_code": "J5R4+57 Irvine, CA, USA",
        "global_code": "8554J5R4+57"
    },
    "results": [
        {
            "address_components": [
                {
                    "long_name": "10",
                    "short_name": "10",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "Curie Court",
                    "short_name": "Curie Ct",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "Irvine",
                    "short_name": "Irvine",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Orange County",
                    "short_name": "Orange County",
                    "types"

In [17]:
uci_address = uci_addr_info["results"][0]["formatted_address"]

In [19]:
#create a dataframe to hold location information
uci_address_df = pd.DataFrame({"Location Name" : location_name
                              ,"Address" : uci_address
                              ,"lat" : uci_lat
                              ,"lng" : uci_lng
                              }, index=[0])

uci_address_df

,Location Name,Address,lat,lng
0,University of California Irvine,"10 Curie Ct, Irvine, CA 92617, USA",33.640495,-117.844296


In [26]:
#store results in variables 
ulat = uci_address_df['lat']
ulng = uci_address_df['lng']
uname = uci_address_df['Location Name']
uaddress = uci_address_df['Address']

#define some business search variables
bus_term = 'Restaurants'
bus_location = 'Irvine, California'
bus_limit = 10
#set the search radius equal to 2 miles around the location
bus_radius = milestometers(2)


In [28]:
#get yelp business info with above requirements
yelp_info = getyelpbusiness(bus_term,bus_location,ulat,ulng,bus_limit,bus_radius,YELP_API_KEY)

In [29]:
print(json.dumps(yelp_info, indent=4, sort_keys=True))

{
    "businesses": [
        {
            "alias": "north-italia-irvine",
            "categories": [
                {
                    "alias": "pizza",
                    "title": "Pizza"
                },
                {
                    "alias": "italian",
                    "title": "Italian"
                }
            ],
            "coordinates": {
                "latitude": 33.67179,
                "longitude": -117.84507
            },
            "display_phone": "(949) 629-7060",
            "distance": 3477.2791238865525,
            "id": "NVB2X8t5Rie8S5AnzOnWJg",
            "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/TWL29M9UmLCIE1BZrXni8A/o.jpg",
            "is_closed": false,
            "location": {
                "address1": "2957 Michelson Dr",
                "address2": "",
                "address3": "",
                "city": "Irvine",
                "country": "US",
                "display_address": [
                    "2957

In [31]:
#view the main parts of the api get call
for x in yelp_info:
    print(x)
    
#from this we only want to know the details from the [businesses] item

businesses
total
region


In [32]:
#pull the data from the yelp api business call and build a list containing a dictionary object
restaurants = []
rest_info = {}

for restaurant in yelp_info["businesses"]:
    #print(restaurant["name"])
    
    #rest_type = restaurant["categories"][0]
    #rest_subtype = restaurant["categories"]
    
    #print(f'{rest_name} - {rest_type} - {rest_subtype}')   
    #print(f'{rest_subtype[0]}')   
    
    rest_name = restaurant["name"]
    
    #account for the number of possible categories, we only want a max of 2
    numlen = len(restaurant["categories"])
    #print(numlen)
    if numlen <1:
        rest_type = "n/a"
        rest_subtype = "n/a"
    elif numlen == 1:
        rest_type = restaurant["categories"][0]["title"]
        rest_subtype = "n/a"
    elif numlen > 1:
        rest_type = restaurant["categories"][0]["title"]
        rest_subtype = restaurant["categories"][1]["title"]
    else:
        rest_type = "n/a"
        rest_subtype = "n/a"
    
    rest_lat = restaurant["coordinates"]["latitude"]
    rest_lng = restaurant["coordinates"]["longitude"]
    
    rest_distance = restaurant["distance"]   
    #make sure its not zero
    #save meters and miles
    if rest_distance != None or rest_distance ==0:
        rest_distance_meters = int(rest_distance)
        rest_distance_miles = meterstomiles(rest_distance)

    else:
        rest_distance_meters = 0
        rest_distance_miles = 0
     
    rest_id = restaurant["id"]
    rest_addr1 = restaurant["location"]["address1"]
    rest_addr2 = restaurant["location"]["address2"]
    rest_city = restaurant["location"]["city"]
    rest_country = restaurant["location"]["country"]
    rest_state = restaurant["location"]["state"]
    rest_zip = restaurant["location"]["zip_code"]
    rest_price = restaurant["price"]
    rest_rating = restaurant["rating"]
    rest_review = restaurant["review_count"]
    
    rest_info = {"yelpid":rest_id
                ,"name":rest_name
                ,"lat":rest_lat
                ,"lng":rest_lng
                ,"distance meters": rest_distance_meters
                ,"distance miles": rest_distance_miles
                ,"address1":rest_addr1
                ,"address2":rest_addr2
                ,"city":rest_city
                ,"state":rest_state
                ,"zip":rest_zip
                ,"price rating":rest_price
                ,"review rating":rest_rating
                ,"review count": rest_review
                ,"type":rest_type
                ,"sub type":rest_subtype
                }
    restaurants.append(rest_info)
    
    print(rest_addr1)
    #print(restaurant["location"])
    #print(rest_type)
    #rest_subtype

2957 Michelson Dr
5171 California Ave
1332 Bison Ave
1370 Bison Ave
4647 MacArthur Blvd
4515 Campus Dr
4503 Campus Dr
4249 Campus Dr
2301 San Joaquin Hills Rd
4143 Campus Dr


In [33]:
#build the dataframe and reorder the columns
restaurant_df = pd.DataFrame(restaurants)
restaurant_df = restaurant_df[['yelpid','name','lat','lng','distance miles','distance meters'
                               ,'price rating','review count', 'review rating'
                              ,'type','sub type','address1','city','state','zip']]
#.set_index('id',[0])
restaurant_df = restaurant_df.reset_index()
restaurant_df.head()

,index,yelpid,name,lat,lng,distance miles,distance meters,price rating,review count,review rating,type,sub type,address1,city,state,zip
0,0,NVB2X8t5Rie8S5AnzOnWJg,North Italia,33.671790,-117.845070,2.16,3477,$$,2209,4.5,Pizza,Italian,2957 Michelson Dr,Irvine,CA,92612
1,1,Ut9709JWjLlx9CYyTjYYKg,Thai Style,33.640992,-117.855753,0.66,1062,$$,4,4.5,Thai,Food Stands,5171 California Ave,Irvine,CA,92617
2,2,UyQiOlHYL6AjMrr_Q8ty1Q,The Stand,33.637546,-117.860111,0.92,1477,$$,185,4.5,American (New),Burgers,1332 Bison Ave,Newport Beach,CA,92660
3,3,kgR9KrpmqrwJeKmyKnEA0Q,Eddie V's Prime Seafood,33.637768,-117.861074,0.98,1582,$$$,1334,4.5,Seafood,Steakhouses,1370 Bison Ave,Newport Beach,CA,92660
4,4,ObCj8Y3KAdk4bgc8oSxNcA,Bosscat Kitchen and Libations,33.669850,-117.865260,2.38,3823,$$,2493,4.5,American (New),Lounges,4647 MacArthur Blvd,Newport Beach,CA,92660


In [34]:
#check the columns
restaurant_df.columns

Index(['index', 'yelpid', 'name', 'lat', 'lng', 'distance miles',
       'distance meters', 'price rating', 'review count', 'review rating',
       'type', 'sub type', 'address1', 'city', 'state', 'zip'],
      dtype='object')

In [119]:
#set a variable to loop through the businesses by id
#yelpid_data = restaurant_df["yelpid"]
#print(yelpid_data)

#setup some variables for the reviews
yelp_reviews = []
reviews_info = {}

In [126]:
#-----VADER-->
vader_results = []
#<----END VADER


for index, row in restaurant_df.iterrows():
        
    #-----VADER-->
    com_ls = []
    pos_ls = []
    neg_ls = []
    neu_ls = []
    #<----END VADER
    
    #iterate through df
    
    #get the id
    yid = row[1]
    #print(f'{index}: {yid}')
    #restaurant name
    yname = row[2]
    
    #call our yelpfunc and sleep it for 2 seconds
    yreview = getyelpreview(yid,YELP_API_KEY)
    #print(yreview)
    #time.sleep(2)
    
    for review in yreview["reviews"]:
        
        #the restaurant id we will be using to join on later
        rest_id = yid
        
        ls_review_id = review["id"]
        ls_review_rating = review["rating"]
        ls_review_text = review["text"]
        ls_review_timestamp = review["time_created"]
        
        #add yelp review name and review id
        ls_review_name = review["user"]["name"]
        ls_review_reviewerid = review["user"]["id"]
        
        
        #check to see if we have text to parse
        #print(ls_review_text)
        
        #--->VADER
        
        #run vader analysis on each text
        results = analyzer.polarity_scores(ls_review_text)
        com = results["compound"]
        pos = results["pos"]
        neg = results["neg"]
        neu = results["neu"]
       
        #add each value to each list
        com_ls.append(com)
        pos_ls.append(pos)
        neg_ls.append(neg)
        neu_ls.append(neu)
        #--------END VADER
        

        reviews_info = {"yelpid":rest_id
                        ,"reviewid": ls_review_id
                        ,"review text": ls_review_text
                        ,"timestamp": ls_review_timestamp
                        ,"reviewer name": ls_review_name
                        ,"reviewer id": ls_review_reviewerid
                        ,"compound" : com
                        ,"positive" : pos
                        ,"negative" : neg
                        ,"neutral" : neu
                       }
        yelp_reviews.append(reviews_info)
        
    #sentiment results
    user_results = {"Restaurant": yname
                    ,"Compound Score": np.mean(com_ls)
                    ,"Postive Score": np.mean(pos_ls)
                    ,"Negative Score": np.mean(neg_ls)
                    ,"Neutral Score": np.mean(neu_ls)
                   }
    vader_results.append(user_results)
        
print(f'Compound: {com_ls} or {com}')


Compound: [0.8685, 0.3716, 0.9325] or 0.9325
Restaurant: Eureka!
Compound: 0.724
Positive: 0.263
Neutral: 0.717
Negative: 0.020


In [130]:
#view the vader results
#print(vader_results)

#create a sentiment dataframe
sentiments_pd = pd.DataFrame.from_dict(vader_results)
sentiments_pd

,Compound Score,Negative Score,Neutral Score,Postive Score,Restaurant
0,0.384000,0.041667,0.819000,0.139333,North Italia
1,0.725333,0.021333,0.758667,0.220333,Thai Style
2,0.000000,0.043667,0.912333,0.043667,The Stand
3,0.647333,0.000000,0.757333,0.242667,Eddie V's Prime Seafood
4,0.594633,0.014000,0.837000,0.149333,Bosscat Kitchen and Libations
5,0.408433,0.044667,0.816333,0.139000,Hen House Grill
6,0.173267,0.109333,0.746000,0.144667,Asian Box
7,0.206333,0.025000,0.862333,0.112667,Slapfish
8,0.432800,0.000000,0.875000,0.125000,Farmhouse At Roger's Gardens
9,0.724200,0.019667,0.717333,0.263000,Eureka!


In [117]:
#clear the results for testing
reviews_df = None
yelp_reviews = None
#yelp_reviews

In [132]:
#build the dataframe and reorder the columns
reviews_df = pd.DataFrame(yelp_reviews)
reviews_df.head()

,compound,negative,neutral,positive,review text,reviewer id,reviewer name,reviewid,timestamp,yelpid
0,0.8074,0.000,0.751,0.249,"Not only the amazing food, but after asking if...",S9_npMJoctCGmg61ZBuYjA,Jason P.,QLpop9_8J1_SahYyz25F5w,2018-09-09 19:10:19,NVB2X8t5Rie8S5AnzOnWJg
1,0.0000,0.000,1.000,0.000,We were staying across the way at the AC Hotel...,5XAXkk6WENi0OW_HSSXRWA,Chuck D.,bc-20X2-fnbly5UBqJvG3A,2018-09-12 12:10:43,NVB2X8t5Rie8S5AnzOnWJg
2,0.3446,0.125,0.706,0.169,"I honestly like this place a lot, so I'm upset...",QYVPF8N9lpTPQcBKsZ8Hpw,Rulla A.,or8zUCgQhvqGa3g0433dmQ,2018-09-12 00:23:53,NVB2X8t5Rie8S5AnzOnWJg
3,0.7506,0.064,0.669,0.268,Good weekly Thai food @our Farmer's Market on ...,-2CainXEr63oltxnExHGgg,Randy W.,iP-ARrjqnobZSf34oQFJpg,2017-07-27 12:21:45,Ut9709JWjLlx9CYyTjYYKg
4,0.6542,0.000,0.840,0.160,I agree with other yelpers saying the wait can...,Nl7_kl_3QbEexPFud0xobg,An L.,_odtz8ELynWfQXTdT1oc3Q,2018-03-01 15:43:14,Ut9709JWjLlx9CYyTjYYKg


In [136]:
#view column info
reviews_df.columns

Index(['index', 'yelpid', 'reviewid', 'compound', 'positive', 'negative',
       'neutral', 'review text', 'timestamp', 'reviewer name', 'reviewer id'],
      dtype='object')

In [147]:
reviews_df = reviews_df[['yelpid','reviewid','compound','positive','negative','neutral','review text','timestamp','reviewer name','reviewer id']]
reviews_df = reviews_df.reset_index()
reviews_df.head(3)

,index,yelpid,reviewid,compound,positive,negative,neutral,review text,timestamp,reviewer name,reviewer id
0,0,NVB2X8t5Rie8S5AnzOnWJg,QLpop9_8J1_SahYyz25F5w,0.8074,0.249,0.000,0.751,"Not only the amazing food, but after asking if...",2018-09-09 19:10:19,Jason P.,S9_npMJoctCGmg61ZBuYjA
1,1,NVB2X8t5Rie8S5AnzOnWJg,bc-20X2-fnbly5UBqJvG3A,0.0000,0.000,0.000,1.000,We were staying across the way at the AC Hotel...,2018-09-12 12:10:43,Chuck D.,5XAXkk6WENi0OW_HSSXRWA
2,2,NVB2X8t5Rie8S5AnzOnWJg,or8zUCgQhvqGa3g0433dmQ,0.3446,0.169,0.125,0.706,"I honestly like this place a lot, so I'm upset...",2018-09-12 00:23:53,Rulla A.,QYVPF8N9lpTPQcBKsZ8Hpw


In [162]:
#merge both dataframes
#https://stackoverflow.com/questions/18792918/combine-two-pandas-data-frames-join-on-a-common-column
#pd.merge(restaurant_ids_dataframe, restaurant_review_frame, on='business_id', how='outer')
#pd.merge(restaurant_ids_dataframe, restaurant_review_frame, on='business_id', how='outer', suffixes=('_restaurant_id', '_restaurant_review'))

newdf = pd.merge(restaurant_df, reviews_df, on='yelpid', how="outer")

#removing merged indexes
del newdf["index_x"]
del newdf["index_y"]

#setting the df to our cleaned df
newdf = newdf

#resetting the index
newdf = newdf.reset_index(drop=True)

newdf.head(1)
#new_df = None

,yelpid,name,lat,lng,distance miles,distance meters,price rating,review count,review rating,type,...,zip,reviewid,compound,positive,negative,neutral,review text,timestamp,reviewer name,reviewer id
0,NVB2X8t5Rie8S5AnzOnWJg,North Italia,33.67179,-117.84507,2.16,3477,$$,2209,4.5,Pizza,...,92612,QLpop9_8J1_SahYyz25F5w,0.8074,0.249,0.0,0.751,"Not only the amazing food, but after asking if...",2018-09-09 19:10:19,Jason P.,S9_npMJoctCGmg61ZBuYjA


In [163]:
#get the columns
newdf.columns

Index(['yelpid', 'name', 'lat', 'lng', 'distance miles', 'distance meters',
       'price rating', 'review count', 'review rating', 'type', 'sub type',
       'address1', 'city', 'state', 'zip', 'reviewid', 'compound', 'positive',
       'negative', 'neutral', 'review text', 'timestamp', 'reviewer name',
       'reviewer id'],
      dtype='object')

In [165]:
#define unique restauraunts
restsnum = newdf.index.unique()
print(f'{len(restsnum)} restsnum \n{restsnum}')

180 restsnum 
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            170, 171, 172, 173, 174, 175, 176, 177, 178, 179],
           dtype='int64', length=180)
